In [1]:
import glob
import os

import numpy as np
import pandas as pd

# setting notebook
pd.set_option("display.max_columns", None)

In [2]:
# go to data-joined folder
%cd data-joined

C:\Users\user\Desktop\gte\notebooks\data-joined


In [3]:
%run xuat_combined.ipynb 
%run po_combined.ipynb 
%run dmsp.ipynb
%run dmkh.ipynb
#go back the tree
%cd ..

C:\Users\user\Desktop\gte\notebooks


In [4]:
# make copy
xuat = xuat_joined.copy()
po = po_joined.copy()
sp = dmsp_full.copy()
kh = dmkh_full.copy()

## PO

In [5]:
#drop cols
cols_used = ['Mã phiếu đề xuất\n(Code)',
 'Mã khách hàng\n(Customer Code)',
 'Mã chủ hàng',
 'Ngày về\n(Receiving Date)',
 'Hạn chót giao hàng\n(Delivery Deadline)',
 'Ngày thực tế giao hàng',
 'Chứng từ KH ký nhận',
 'Chứng từ khác',
 'Tình trạng đơn hàng',
 'Kho xuất',
 'Loại phiếu',
 'Người giao hàng (Shipper delivery by)',
'Unnamed: 23']

po_selected_cols = po[cols_used]

In [6]:
po_replace = po_selected_cols.copy()

#Replace 'Chua thuc hien' => 'Da nhat hang'
po_replace.loc[(po_replace['Tình trạng đơn hàng'].str.contains('Chưa thực hiện')) 
                & (po_replace['Chứng từ khác'].str.contains('Đã nhặt hàng')),'Tình trạng đơn hàng'] = 'Đã nhặt hàng'

#Replace 'De xuat' => 'Don'
po_replace['Loại phiếu'] = po_replace['Loại phiếu'].str.replace(r'Đề xuất', 'Đơn', regex=True)

In [7]:
# make a copy
po_cal = po_replace.copy()
# create month
po_cal['month'] = pd.DatetimeIndex(po_cal['Ngày về\n(Receiving Date)']).month
# impute date diff invoice

diff_ct_yes = (po_cal['Chứng từ KH ký nhận'] - po_cal['Ngày thực tế giao hàng']).dt.days #da hoan
diff_ct_no = (pd.Timestamp.now().normalize() - po_cal['Ngày thực tế giao hàng']).dt.days #chua hoan

conditions = [
    po_cal['Tình trạng đơn hàng']=='Đã bàn giao chứng từ',
    po_cal['Tình trạng đơn hàng']=='Đã giao hàng'
]
choices = [diff_ct_yes, diff_ct_no]

po_cal['datediff_chungtu'] = np.select(conditions, choices, default=np.nan)

# impute date diff delivery

diff_deli = (po_cal['Ngày thực tế giao hàng'] - po_cal['Ngày về\n(Receiving Date)']).dt.days
po_cal['datediff_giaohang'] = np.where(diff_deli<0, np.nan, diff_deli)

In [8]:
po_filter = po_cal.copy()
#Filter Warehouse not contains
values_to_remove = ['Hyperion','hư hỏng','xử lý','Hư Hỏng','khuyến mãi','Tồn hàng','hàng hóa', 'Kho treo hàng']

pattern = '|'.join(values_to_remove)

po_filter_kho = po_filter.loc[~po_filter['Kho xuất'].str.contains(pattern, case=False)]
#Filter Ma chu hang HP, RW
values_to_remove_mch = ['HP', 'RW', 'GB', 'HP01', 'RW01'] 

pattern_mch = '|'.join(values_to_remove_mch)

po_filter_mch = po_filter_kho.loc[po_filter_kho['Mã chủ hàng'].str.contains(pattern_mch, case=False)]
po_processed = po_filter_mch.copy()

## BCKT

In [9]:
# merge
xuat_dmsp = xuat.merge(sp, left_on='Mã vật tư\n(Goods Code)', right_on='PRODUCT CODE', how='left')
xuat_dmsp['CBM/Unit'] = xuat_dmsp['CBM/Unit']*xuat_dmsp['Số lượng\n(Quantily)']
xuat_dmsp['GrossWeightProductg'] = xuat_dmsp['GrossWeightProductg']*xuat_dmsp['Số lượng\n(Quantily)']
# aggregate 
xuat_dmsp = xuat_dmsp.groupby(['Mã phiếu đề xuất', 'Mã chủ hàng_x']).agg(
      {
        'CBM/Unit': 'sum',
        'GrossWeightProductg': 'sum',
        'Số lượng\n(Quantily)': 'sum',
        'Mã vật tư\n(Goods Code)': 'count',
        'Tiền hàng\n(Sub Total)': 'sum'
      }
)
# rename cols
dict_rename = {
    'CBM/Unit': 'cbm_po',
     'GrossWeightProductg': 'kg_po',
     'Số lượng\n(Quantily)': 'unit_po',
     'Mã vật tư\n(Goods Code)': 'sku_po',
     'Tiền hàng\n(Sub Total)': 'amount_po'
}

xuat_dmsp.rename(columns=dict_rename, inplace=True)
xuat_processed = xuat_dmsp.copy()

## Create data

In [10]:
# Create data region
data = [
    ["Kho Hà Nội", "Miền Bắc"],
    ["Kho Phạm Văn Đồng", "Miền Nam"],
    ["Kho Bình Tân", "Miền Nam"],
    ["Kho Bùi Thị Xuân", "Miền Nam"],
    ["Kho Đà Nẵng", "Miền Trung"],
    ["Kho Thuận An", "Miền Nam"],
    ["Kho Bình Hòa", "Miền Nam"],
    ["Kho thuê dịch vụ - Daso", "Miền Nam"],
    ["Kho Woojin Bắc Ninh", "Miền Bắc"],
    ["Kho LogisValley - Long Biên", "Miền Bắc"],
    ["Kho thuê ngoài Emskip Q9", "Miền Nam"],
    ["Kho LogisticValey - Long Biên", "Miền Bắc"],
    ["Kho Hateco Long Biên", "Miền Bắc"],
    ["Kho Gamuda Hà Nội", "Miền Bắc"],
    ["Kho thuê dịch vụ Daso", "Miền Nam"],
    ['Kho LogisticValey Long Biên',"Miền Bắc"],
    ['Kho Online PVD', 'online'],
    ['Kho Upbase Nơ Trang Long', 'online'],
    ['Kho Online Nơ Trang Long', 'online'],
    ['Kho Online Hà Nội', 'online'],
    ['Kho Online Upbase Hà Nội', 'online'],
    ['Kho Online Upbase PVD', 'online'],
    ['Kho Online Đà Nẵng','online']
]

# Create the pandas DataFrame
df_rgn = pd.DataFrame(data, columns=["Kho", "Region"])
data_in_sub = [
    ["Kho Hà Nội", "Hà Nội", "Nội thành"],
    ["Kho Phạm Văn Đồng", "Hồ Chí Minh", "Nội thành"],
    ["Kho Bình Tân", "Hồ Chí Minh", "Nội thành"],
    ["Kho Bùi Thị Xuân", "Bình Dương", "Nội thành"],
    ["Kho Đà Nẵng", "Đà Nẵng", "Nội thành"],
    ["Kho Thuận An", "Bình Dương", "Nội thành"],
    ["Kho Bình Hòa", "Hồ Chí Minh", "Nội thành"],
    ["Kho thuê dịch vụ - Daso", "Hồ Chí Minh", "Nội thành"],
    ["Kho Woojin Bắc Ninh", "Hà Nội", "Nội thành"],
    ["Kho LogisValley - Long Biên", "Hà Nội", "Nội thành"],
    ["Kho thuê ngoài Emskip Q9", "Hồ Chí Minh", "Nội thành"],
    ["Kho LogisticValey - Long Biên", "Hà Nội", "Nội thành"],
    ["Kho Hateco Long Biên", "Hà Nội", "Nội thành"],
    ["Kho Gamuda Hà Nội", "Hà Nội", "Nội thành"],
    ["Kho thuê dịch vụ Daso", "Hồ Chí Minh", "Nội thành"],
    ['Kho LogisticValey Long Biên',"Hà Nội", "Nội thành"]
]

# Create the pandas DataFrame
df_in_sub = pd.DataFrame(data_in_sub, columns=["Kho xuất", "Des", "in_sub"])

## Merge

In [11]:
# merge po - bckt
po_xuat = po_processed.merge(
    xuat_processed, left_on=['Mã phiếu đề xuất\n(Code)', 'Mã chủ hàng'], right_on=['Mã phiếu đề xuất', 'Mã chủ hàng_x'], how="left")
# merge region
po_region = po_xuat.merge(df_rgn, left_on="Kho xuất", right_on="Kho", how="left").drop('Kho', axis=1)
# merge DMKH
po_kh = po_region.merge(
    kh[['Customer Code', 'Account', 'Des']], left_on="Mã khách hàng\n(Customer Code)", right_on="Customer Code", how="left").drop('Customer Code', axis=1)
# merge inner/suburbs data
po_insub = po_kh.merge(df_in_sub, on=["Kho xuất", "Des"], how="left")
po_insub.loc[~po_insub.Des.isna(), "in_sub"] = po_insub.in_sub.fillna("Ngoại thành")



In [12]:
# day level
conditions = [
    po_insub["datediff_chungtu"] < 0,
    po_insub["datediff_chungtu"] <= 6,
    po_insub["datediff_chungtu"] <= 14,
    po_insub["datediff_chungtu"] <= 30,
    po_insub["datediff_chungtu"] > 30,
]
choices = ["Invalid_days", "0 - 6 ngày", "7 - 14 ngày", "15 - 30 ngày", "Trễ"]

po_insub["day_level"] = np.select(conditions, choices, default=np.nan)

In [13]:
# rename
po_run = po_insub.copy()
po_run['Mã phiếu đề xuất\n(Code)'] = po_run['Mã phiếu đề xuất\n(Code)'] + po_run['Mã chủ hàng']
dict_rename = {
    "Mã phiếu đề xuất\n(Code)": "ID_PO",
    "Mã khách hàng\n(Customer Code)": "ID_cus",
    "Ngày về\n(Receiving Date)": "date_receive",
    "Hạn chót giao hàng\n(Delivery Deadline)": "deli_dl",
    "Tiền hàng\n(Sub Total)": "amount_po",
    'Unnamed: 23':'ngaybangiaohoadon'
}

po_run.rename(columns=dict_rename, inplace=True)

In [14]:
# import datetime
# file_path = '../data/processed/'
# namefile = f'PO_{datetime.datetime.now().strftime("%H%M_%Y%m%d")}.csv'
# po_run.to_csv(file_path+namefile, index=False, encoding="utf-8-sig")


In [15]:
# po_run[po_run.Region.isnull()]['Kho xuất'].unique()

In [16]:
po_run['ngaybangiaohoadon'] = np.where(~po_run['ngaybangiaohoadon'].astype(str).str.contains(r'^\d{2}\/\d{2}\/\d{4}', regex=True, na=False), pd.NaT, po_run['ngaybangiaohoadon'])
po_run['ngaybangiaohoadon'] = po_run['ngaybangiaohoadon'].apply(pd.to_datetime, format='%d/%m/%Y')

## chung tu HP

In [17]:
po_hp = po_run[po_run['Mã chủ hàng'] == 'HP']

In [18]:
pd.pivot_table(po_hp, index='Tình trạng đơn hàng', values='ID_PO', aggfunc='count').reset_index()

,Tình trạng đơn hàng,ID_PO
0,Chưa giao hàng,2070
1,Chưa thực hiện,1027
2,PO hủy,7450
3,Đã bàn giao chứng từ,145538
4,Đã giao hàng,5933
5,Đã nhặt hàng,51


In [19]:
chungtu_chuahoan = po_hp[(po_hp['Tình trạng đơn hàng'] == 'Đã giao hàng')|(po_hp['Tình trạng đơn hàng'] == 'Đã bàn giao chứng từ')]

pd.pivot_table(chungtu_chuahoan, index='day_level', columns='Tình trạng đơn hàng', values='ID_PO', aggfunc='count')


Tình trạng đơn hàng,Đã bàn giao chứng từ,Đã giao hàng
day_level,,
0 - 6 ngày,102615.0,2625.0
15 - 30 ngày,8681.0,535.0
7 - 14 ngày,31936.0,2583.0
Invalid_days,196.0,NaN
Trễ,1616.0,190.0
nan,494.0,NaN


In [20]:

pd.pivot_table(chungtu_chuahoan[(chungtu_chuahoan['Tình trạng đơn hàng'] == 'Đã giao hàng')], index='day_level', columns='Tình trạng đơn hàng', values='amount_po', aggfunc=np.sum).reset_index()

Tình trạng đơn hàng,day_level,Đã giao hàng
0,0 - 6 ngày,1.160058e+10
1,15 - 30 ngày,2.294822e+09
2,7 - 14 ngày,9.175651e+09
3,Trễ,8.871761e+08


In [21]:
pd.pivot_table(chungtu_chuahoan[(chungtu_chuahoan['Tình trạng đơn hàng'] == 'Đã giao hàng')], index='month', columns='Tình trạng đơn hàng', values='amount_po', aggfunc=np.sum)

Tình trạng đơn hàng,Đã giao hàng
month,
1,2.270016e+08
3,3.005845e+07
5,4.100851e+06
7,6.232800e+05
8,2.968000e+05
9,2.173244e+07
10,3.424400e+07
11,1.649290e+09
12,2.199088e+10


In [22]:
pd.pivot_table(chungtu_chuahoan[(chungtu_chuahoan['Tình trạng đơn hàng'] == 'Đã giao hàng')], index='Region', columns='Tình trạng đơn hàng', values='amount_po', aggfunc=np.sum).reset_index()

Tình trạng đơn hàng,Region,Đã giao hàng
0,Miền Bắc,9.714478e+09
1,Miền Nam,1.300342e+10
2,Miền Trung,1.239828e+09
3,online,4.976000e+05


In [23]:
# da hoan chung tu
pd.pivot_table(chungtu_chuahoan[(chungtu_chuahoan['Tình trạng đơn hàng'] == 'Đã bàn giao chứng từ')], index='Region', columns='day_level', values='ID_PO', aggfunc='count').reset_index()

day_level,Region,0 - 6 ngày,15 - 30 ngày,7 - 14 ngày,Invalid_days,Trễ,nan
0,Miền Bắc,39251.0,4553.0,13781.0,124.0,750.0,22.0
1,Miền Nam,55028.0,3372.0,15854.0,43.0,670.0,104.0
2,Miền Trung,7102.0,754.0,2300.0,29.0,194.0,NaN
3,online,1234.0,2.0,1.0,NaN,2.0,368.0


In [24]:
# rate hoan chung tu
rate_chungtu = chungtu_chuahoan.copy()
rate_chungtu['month_ct'] = pd.DatetimeIndex(rate_chungtu['ngaybangiaohoadon']).month


In [25]:
# Define the aggregation calculations
aggregations = {
    'ngaybangiaohoadon': 'count'
}
rate_chungtu[rate_chungtu['Tình trạng đơn hàng']=='Đã bàn giao chứng từ'].groupby(['month_ct'], as_index=False, dropna=False).agg(aggregations)

,month_ct,ngaybangiaohoadon
0,1.0,5328
1,2.0,6279
2,3.0,12962
3,4.0,11872
4,5.0,15320
5,6.0,14038
6,7.0,14708
7,8.0,14938
8,9.0,12386
9,10.0,13853


In [42]:
po_hp['Kho xuất'].unique()

array(['Kho Phạm Văn Đồng', 'Kho Bùi Thị Xuân', 'Kho Gamuda Hà Nội',
       'Kho Bình Tân', 'Kho Đà Nẵng', 'Kho Online PVD', 'Kho Bình Hòa',
       'Kho Thuận An', 'Kho Online Hà Nội', 'Kho thuê dịch vụ Daso',
       'Kho LogisticValey Long Biên', 'Kho Online Nơ Trang Long',
       'Kho Upbase Nơ Trang Long', 'Kho Online Đà Nẵng',
       'Kho Online Upbase Hà Nội', 'Kho Hateco Long Biên',
       'Kho thuê ngoài Emskip Q9', 'Kho Woojin Bắc Ninh'], dtype=object)

## van hanh

In [26]:
po_summa = po_run.copy()

In [27]:
po_summa['Người giao hàng (Shipper delivery by)'] = po_summa['Người giao hàng (Shipper delivery by)'].str.lower()

In [28]:
cond = po_summa['Người giao hàng (Shipper delivery by)'].str.contains('nhất|viet|ker|xe tải ngoài|taixe thau hcm', na=False)
po_summa['dvngoai'] = po_summa.loc[cond, 'Người giao hàng (Shipper delivery by)']

In [29]:
# po_summa.to_csv('../data/processed/po-meeting.csv',encoding='utf-8-sig', index=False)

## load data

In [30]:
# meeting = pd.read_csv('../data/processed/po-meeting.csv', encoding='utf-8-sig')

In [31]:
meeting = po_summa.copy()

In [32]:
meeting = meeting[meeting['month']>=8]

In [33]:
meeting_xuatban = meeting[(meeting['Loại phiếu'] == 'Đơn bán')].reset_index(drop=True)

In [34]:
# Define the aggregation calculations
aggregations = {
    'date_receive': ['count', 'nunique'],
    'amount_po': ['mean', 'median'],
    'cbm_po': 'median',
    'kg_po': 'median',
    'unit_po': 'median',
    'sku_po': 'mean'
}
summa1 = meeting_xuatban.groupby(['Mã chủ hàng', 'Region'], as_index=False, dropna=False).agg(aggregations)

In [35]:
# summa1.columns = ['_'.join(cols).lower()
#                      for cols in summa1.columns.to_flat_index()]

In [36]:
summa1[(summa1['Mã chủ hàng'] == 'GB')|(summa1['Mã chủ hàng'] == 'HP01')|(summa1['Mã chủ hàng'] == 'RW01')]

Mã chủ hàng  Region date_receive              amount_po            cbm_po  \
                              count nunique           mean    median  median   
0           GB  online        11253     142  161214.904648  107639.0  0.0018   
5         HP01  online          921     100  188084.730727  157407.0  0.0030   
10        RW01  online         9044     108  124642.707099   95455.0  0.0016   

     kg_po unit_po    sku_po  
    median  median      mean  
0    332.0     3.0  1.747801  
5   2120.0     2.0  1.555917  
10  1085.0     1.0  1.215170

In [37]:
online = meeting_xuatban[(meeting_xuatban['Tình trạng đơn hàng'] == 'Đã bàn giao chứng từ')|(meeting_xuatban['Tình trạng đơn hàng'] == 'Đã giao hàng')].reset_index(drop=True)

In [38]:
online_fiter_mch = online[(online['Mã chủ hàng'] == 'GB')|(online['Mã chủ hàng'] == 'HP01')|(online['Mã chủ hàng'] == 'RW01')].reset_index()

In [39]:
# create a list of our conditions
conditions = [
    (online_fiter_mch['amount_po'] < 100000),
    (online_fiter_mch['amount_po'] < 200000),
    (online_fiter_mch['amount_po'] < 300000),
    (online_fiter_mch['amount_po'] >= 300000)
    ]

# create a list of the values we want to assign for each condition
values = ['duoi100', '100-200', '200-300', 'tren300']

# create a new column and use np.select to assign values to it using our lists as arguments
online_fiter_mch['amountlevel'] = np.select(conditions, values)

In [40]:
# Define the aggregation calculations
aggregations = {
    'ID_PO': 'count'
}
online_fiter_mch.groupby(['Mã chủ hàng','amountlevel'], as_index=False, dropna=False).agg(aggregations)

,Mã chủ hàng,amountlevel,ID_PO
0,GB,100-200,3554
1,GB,200-300,1109
2,GB,duoi100,5001
3,GB,tren300,1218
4,HP01,100-200,304
5,HP01,200-300,143
6,HP01,duoi100,345
7,HP01,tren300,109
8,RW01,100-200,3454
9,RW01,200-300,606
